<a href="https://colab.research.google.com/github/inokchoi/Speech/blob/main/dvector_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# d-vector Experiments

## Import packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
cd '/content/drive/My Drive/Colab Notebooks/20-1voice_interface/Proj2/dvector/'

/content/drive/My Drive/Colab Notebooks/20-1voice_interface/Proj2/dvector


In [ ]:
#import glob

In [ ]:
#train_list = glob.glob('../wsj0/*/*.wav')

In [ ]:
#train_list

In [ ]:
'''
#speaker_list = [train_list[i].split('/')[-1].split('\\')[-1].split('_')[0] for i in range(len(train_list))]
speaker_list = [train_list[i][2:] for i in range(len(train_list))]
s = set([])
test_list = []
with open('wsj_train1.txt','w') as f:
    count = 0
    for spk in speaker_list:
        if count % 14 == 0:
          s.add(spk.split('/')[2])
          test_list.append(str(len(s)-1) + ' ' + spk+'\n')
        else:
          s.add(spk.split('/')[2])
          f.write(str(len(s)-1) + ' ' + spk+'\n')
        count += 1

with open('wsj_test1.txt','w') as f:
    for spk in test_list:
        f.write(spk)

'''

"\n#speaker_list = [train_list[i].split('/')[-1].split('\\')[-1].split('_')[0] for i in range(len(train_list))]\nspeaker_list = [train_list[i][2:] for i in range(len(train_list))]\ns = set([])\ntest_list = []\nwith open('wsj_train1.txt','w') as f:\n    count = 0\n    for spk in speaker_list:\n        if count % 14 == 0:\n          s.add(spk.split('/')[2])\n          test_list.append(str(len(s)-1) + ' ' + spk+'\n')\n        else:\n          s.add(spk.split('/')[2])\n          f.write(str(len(s)-1) + ' ' + spk+'\n')\n        count += 1\n\nwith open('wsj_test1.txt','w') as f:\n    for spk in test_list:\n        f.write(spk)\n\n"

In [ ]:
#speaker_list = [test_list[i][2:] for i in range(len(test_list))]
#s = set([])
#with open('wsj_test1.txt','w') as f:
#    for spk in speaker_list:
#        s.add(spk.split('/')[2])
#        f.write(str(len(s)-1) + ' ' + spk+'\n')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm #프린트 예쁘게
import numpy as np
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from PJ4_dataset import *
from PJ4_model import *
import os
import logging

## Training

In [ ]:
# Define a train function.
def train(epoch, model, criterion, optimizer, dataloader, device):
    model.train()
    bar = tqdm(enumerate(dataloader))
    
    samples = 0
    total_loss, total_acc =0, 0
    for batch_idx, (data,label) in bar:
        
        data = data.type(torch.FloatTensor).to(device)
        label = label.to(device)
        
        # Pass the input data through the defined network architecture.
        pred = model(data, extract=True)

        # Compute a loss function.
        loss = criterion(pred, label) #크로스엔트로피로 로스를 측정
        total_loss += loss.item() * len(label) #로스 더하고
        
        acc = torch.sum(torch.eq(torch.argmax(pred,-1), label)).item()
        samples += len(label)
        total_acc += acc
        
        # Perform backpropagation to update network parameters.
        optimizer.zero_grad()
        loss.backward() #백프로파게이션
        optimizer.step() #웨이트 업데이트
        
        bar.set_description('Epoch:{:3d} [{}/{} {:.2f}%] CE Loss: {:.3f} ACC: {:.2f}'.format(
                            epoch, batch_idx, len(dataloader), 100.*(batch_idx/len(dataloader)),
                            total_loss / samples, (total_acc / samples)*100.))   

## Test

In [ ]:
# Define a test function.
def test(model, criterion, dataloader, device):
    model.eval()
    bar = tqdm(dataloader)
    
    samples = 0
    total_loss, total_acc =0, 0
    for batch_idx, (data,label) in enumerate(bar):        
        
        data = data.type(torch.FloatTensor).to(device)
        label = label.to(device)        
        
        # Pass the input data through the defined network architecture.
        pred = model(data, extract=True)       
        
        # Compute a loss function.
        loss = criterion(pred, label)
        total_loss += loss.item()*len(label)
        
        samples += len(label)
        acc = torch.sum(torch.eq(torch.argmax(pred,-1),label)).item()
        total_acc += acc
        
#         """
#         bar.set_description('Epoch:{:3d} [{}/{} {:.2f}%] CE Loss: {:.3f} ACC: {:.2f}'.format(
#                                 epoch, batch_idx,len(dataloader), 100.*(batch_idx/len(dataloader)),
#                                 total_loss / samples,
#                                 (total_acc / samples)*100.))
#         """
    return total_loss/samples, (total_acc/samples)*100.

## DataLoader

In [ ]:
# Define a collate (stacking) function for the data loader.
def collate_fn(samples):
    data, label = [], []
    min_length = min([len(d[0]) for d in samples])-1

    for d, l in samples:
        st = np.random.randint(len(d)-min_length)
        data.append(torch.tensor(d[st:st+min_length]).unsqueeze(0))
        label.append(torch.tensor(l))
    data = torch.cat(data, 0)
    label = torch.LongTensor(label)

    return data, label

# Obtain the dataset and dataloader of train and test data.
def get_dataloader(train_path, test_path, data_path, feature_type ='mel', n_coeff=64):
    train_dataset = SpeakerDataset(train_path, data_path, 
                                   feature_type=feature_type, n_coeff=n_coeff)
    test_dataset = SpeakerDataset(test_path, data_path, 
                                  feature_type=feature_type, n_coeff=n_coeff)
    
    train_loader = DataLoader(train_dataset, 
                              batch_size=8, 
                              shuffle=True, 
                              collate_fn=collate_fn,
                              num_workers=0, 
                              pin_memory=True)
    test_loader = DataLoader(test_dataset, 
                             batch_size=1, 
                             shuffle=False, 
                             collate_fn=collate_fn,
                             num_workers=0, 
                             pin_memory=True)
    
    return train_loader, test_loader

## Main Function
    Define Settings
    Get Dataset
    Get model
    (Load model)
    Train and Validation

In [ ]:
def main(model_path=None):
    ########################################### Settings ##############################################
    # Set the configuration for training.
    epochs = 1000            # number of epochs
    lr = 0.01                # initial learning rate
    n_spk = 29              # number of speakers in the dataset    #스피커가 몇명이냐
    log_path = 'dvector.log' # log file   
    feature_type = 'mel'    # input feature type  #어떤 피쳐
    n_coeff = 13             # feature dimension    #차수
    indim = n_coeff*3        # input dimension (MFCC, delta, delta-delta) #시간축 변화량도 쓴다. 그래서 총 39차원
    context_len = 10         # number of context window  #몇개의 프레임을 합쳐서 입력을 주겠다. (10개 프레임, 아래 코드 보면 10ms 씩 10개 : 100ms)
    outdim = 512             # d-vector output dimension
    #config 파일 만들고 읽는게 더 좋다.
  
    
    # Check if we can use a GPU device.
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    # Define the directory path of training and test datasets. 
    train_path = './wsj_train1.txt' #0~28
    test_path = './wsj_test1.txt' #0~28
    data_path = '..' #트레이닝한 데이터
    
    ############################### Dataset and dataloader #############################################
    train_loader, test_loader = get_dataloader(train_path, test_path, data_path, 
                                               feature_type=feature_type, n_coeff=n_coeff)
      
    #################################      Get model       #############################################

    # Define a network model.
    model = Dvector(n_spk, indim * context_len, outdim).to(device)
    
    # Set the optimizer with Adam.
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # Set the training criterion.
    softmax_criterion = nn.CrossEntropyLoss(reduction='mean')   #한사람만 나오니까 cross entropy loss

    #################################### Load pre-trained model ########################################
    start = 0
    # Load the pre-trained model., 트레이닝된 모델이 있으면 거기부터 트레이닝을 시작한다.
    print('Directory of the pre-trained model: {}'.format(model_path))
    if model_path:
        check = torch.load(model_path)
        start = check['epoch']
        model.load_state_dict(check['model'])
        optimizer.load_state_dict(check['optimizer'])
        print('## Successfully load the model at {} epochs!'.format(start))        
                                                            
    ####################################      Train and Test     ######################################
    prev_loss = 10000
    ct_dec = 0

    for epoch in range(start, epochs+1):

        # Train the network.
        train(epoch, model, softmax_criterion, optimizer, train_loader, device)
        
        # Test the network.
        opt_loss, opt_acc = test(model, softmax_criterion, test_loader,device)
        
        # Print out the results.
#        print("Epoch: {} Test Loss: {:.3f} Test ACC: {:.2f}".format(epoch,opt_loss,opt_acc ))
        
        # Save the optimal model.
        if opt_loss < prev_loss:
            prev_loss = opt_loss            
            torch.save({'epoch': epoch,
                        'model': model.state_dict(),
                        'optimizer': optimizer.state_dict()},
                        './model/model_opt_mel.pth') #토치 모델 저장하는 포맷이다.
            ct_edec = 0
        else:
            ct_dec += 1
            
            # Decrease the learning rate by 2 when the test loss decreases 3 times in a row.
            if ct_dec == 3: #3번 이상 로스가 떨어지지 않으면 러닝 레이트를 반으로 줄인다. (트레이닝 트릭)
                optim_state = optimizer.state_dict()
                optim_state['param_groups'][0]['lr'] /= 2 #러닝레이트 고정하지 않고, 로스펑션을 가져와서 실행한다.
                optimizer.load_state_dict(optim_state)
                print('lr is divided by 2.')
                ct_dec = 0

In [ ]:
main(model_path='./model/model_opt_mel.pth')  #이전에 만든 모델부터 시작해서 계속 트레이닝을 해나간다.
#main()

Directory of the pre-trained model: ./model/model_opt_mel.pth


0it [00:00, ?it/s]

## Successfully load the model at 0 epochs!


Epoch:  0 [487/488 99.80%] CE Loss: 5.828 ACC: 4.36: : 488it [49:21,  6.07s/it]
100%|██████████| 300/300 [03:51<00:00,  1.30it/s]
Epoch:  1 [487/488 99.80%] CE Loss: 4.957 ACC: 4.23: : 488it [15:42,  1.93s/it]
100%|██████████| 300/300 [01:14<00:00,  4.02it/s]
Epoch:  2 [487/488 99.80%] CE Loss: 4.552 ACC: 4.69: : 488it [15:42,  1.93s/it]
100%|██████████| 300/300 [01:15<00:00,  3.99it/s]
Epoch:  3 [487/488 99.80%] CE Loss: 4.152 ACC: 6.18: : 488it [15:37,  1.92s/it]
100%|██████████| 300/300 [01:14<00:00,  4.00it/s]
Epoch:  4 [487/488 99.80%] CE Loss: 4.012 ACC: 6.77: : 488it [15:36,  1.92s/it]
100%|██████████| 300/300 [01:14<00:00,  4.01it/s]
Epoch:  5 [487/488 99.80%] CE Loss: 3.526 ACC: 11.03: : 488it [15:34,  1.92s/it]
100%|██████████| 300/300 [01:14<00:00,  4.03it/s]
Epoch:  6 [487/488 99.80%] CE Loss: 3.642 ACC: 12.57: : 488it [15:33,  1.91s/it]
100%|██████████| 300/300 [01:14<00:00,  4.05it/s]
Epoch:  7 [487/488 99.80%] CE Loss: 3.443 ACC: 16.52: : 488it [15:35,  1.92s/it]
100%|██

lr is divided by 2.


...

Epoch: 17 [487/488 99.80%] CE Loss: 1.049 ACC: 70.22: : 488it [15:36,  1.92s/it]
100%|██████████| 300/300 [01:14<00:00,  4.03it/s]
Epoch: 18 [487/488 99.80%] CE Loss: 0.908 ACC: 72.93: : 488it [15:33,  1.91s/it]
100%|██████████| 300/300 [01:14<00:00,  4.01it/s]
Epoch: 19 [487/488 99.80%] CE Loss: 1.016 ACC: 70.55: : 488it [15:31,  1.91s/it]
100%|██████████| 300/300 [01:14<00:00,  4.04it/s]
Epoch: 20 [487/488 99.80%] CE Loss: 1.145 ACC: 68.32: : 488it [15:30,  1.91s/it]
100%|██████████| 300/300 [01:14<00:00,  4.05it/s]
Epoch: 21 [487/488 99.80%] CE Loss: 1.166 ACC: 67.86: : 488it [15:32,  1.91s/it]
100%|██████████| 300/300 [01:14<00:00,  4.05it/s]
Epoch: 22 [487/488 99.80%] CE Loss: 1.364 ACC: 64.73: : 488it [15:21,  1.89s/it]
100%|██████████| 300/300 [01:13<00:00,  4.10it/s]
0it [00:00, ?it/s]

lr is divided by 2.


Epoch: 23 [487/488 99.80%] CE Loss: 0.650 ACC: 81.89: : 488it [15:06,  1.86s/it]
100%|██████████| 300/300 [01:12<00:00,  4.12it/s]
Epoch: 24 [487/488 99.80%] CE Loss: 0.608 ACC: 83.30: : 488it [15:10,  1.87s/it]
100%|██████████| 300/300 [01:11<00:00,  4.18it/s]
Epoch: 25 [487/488 99.80%] CE Loss: 0.642 ACC: 82.63: : 488it [15:01,  1.85s/it]
100%|██████████| 300/300 [01:12<00:00,  4.17it/s]
0it [00:00, ?it/s]

lr is divided by 2.


Epoch: 26 [487/488 99.80%] CE Loss: 0.418 ACC: 89.23: : 488it [15:01,  1.85s/it]
100%|██████████| 300/300 [01:11<00:00,  4.17it/s]
Epoch: 27 [487/488 99.80%] CE Loss: 0.430 ACC: 89.05: : 488it [15:01,  1.85s/it]
100%|██████████| 300/300 [01:12<00:00,  4.13it/s]
Epoch: 28 [487/488 99.80%] CE Loss: 0.449 ACC: 88.69: : 488it [15:01,  1.85s/it]
100%|██████████| 300/300 [01:11<00:00,  4.19it/s]
Epoch: 29 [487/488 99.80%] CE Loss: 0.428 ACC: 88.79: : 488it [14:58,  1.84s/it]
100%|██████████| 300/300 [01:12<00:00,  4.16it/s]
Epoch: 30 [71/488 14.55%] CE Loss: 0.419 ACC: 88.19: : 72it [02:12,  1.83s/it]